# Future: theta-dependent placefields: build separate placefields for each phase of theta (binned in theta). There should be one set (where the animal is representing the present) that nearly perfectly predicts the animal's location.
    # the rest of the variability 

    1. Basic Hilbert transform
    2. But Theta wave-shape (sawtooth) at higher running speeds.
        - do peak-to-trough and trough-to-peak separate
        ** Nat will send me something
        
- remember Eloy's theta-dependent placefields. I'm ashamed that I fucked up with Eloy.


https://github.com/diba-lab/ephys/blob/master/Analysis/python/LFP/scripts/theta_phase_stim_verify.py
Nat's code for detecting the sawtooth theta is here (lines 271-393ish): https://github.com/diba-lab/ephys/blob/master/Analysis/python/LFP/scripts/theta_phase_stim_verify.py

It's all based on this paper: https://www.jneurosci.org/content/32/2/423

In [ ]:
## Scratchpad for opto
import matplotlib.pyplot as plt
import numpy as np
import Analysis.python.LFP.preprocess_data as pd

import scipy.signal as signal
import pickle
import os
import Analysis.python.LFP.helpers as helpers

In [4]:
## LFP analysis functions from https://github.com/diba-lab/ephys/blob/master/Analysis/python/LFP/lfp_analysis.py
import scipy.signal as signal
import numpy as np

# instead of `import Analysis.python.LFP.lfp_analysis as lfp`
class lfp(object):
    ## Create Butterworth filter - copied from scipy-cookbook webpage
    @staticmethod
    def butter_bandpass(lowcut, highcut, fs, order=2):
        """
        Simplify inputs for creating a Butterworth filter. copied from scipy-cookbook webpage.
        :param lowcut: Hz
        :param highcut: Hz
        :param fs: Sampling rate in Hz
        :param order: (optional) 2 = default
        :return:
        """
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')

        return b, a


    ## filter data through butterworth filter
    @staticmethod
    def butter_bandpass_filter(data, lowcut, highcut, fs, type='filtfilt', order=2):
        """
        Filter data through butterworth bandpass filter. Copied from scipy-cookbook webpage.
        :param data: array of data sampled at fs
        :param lowcut: 4
        :param highcut: 10
        :param type: 'filtfilt' (default) filters both ways, 'lfilt' filters forward only (and likely induces a phase offset).
        :param fs: 30000
        :param order: (optional) default = 2 to match Sieglie et al., eLife (2014)
        :return: filt_data: filtered data
        """
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        if type == 'lfilt':
            filt_data = signal.lfilter(b, a, data)
        elif type == 'filtfilt':
            filt_data = signal.filtfilt(b, a, data)

        return filt_data


    ## Peak-trough detection via Belluscio et al. (2012) J. Neuro
    @staticmethod
    def get_local_extrema(trace, type='max'):
        """ Get local extrema, assuming it occurs near the middle of the trace. spits out an np.nan if there a relative min
        or max occurs at the edge.
        :param trace: lfp trace
        :param type: 'max' (default) or 'min'
        :return: index in trace where max/min is located. np.nan if there is a relative minima/maxima at edge of trace.
        """
        if type == 'max':
            temp = signal.argrelmax(trace, order=int(len(trace)/2))[0]
        elif type == 'min':
            temp = signal.argrelmin(trace, order=int(len(trace)/2))[0]

        if temp.size == 1:
            ind_rel_extreme = temp[0]
        else:
            ind_rel_extreme = np.nan

        return ind_rel_extreme


## Plot trace in a nice working window

In [4]:
chan_plot = 19  # channel you triggered off of
artifact_chan = 13  # this channel should have good stimulation artifact on it for reference...

trace = traces_ds[plot_bool, chan_plot]
trace_lfilt = lfp.butter_bandpass_filter(trace, lowcut, highcut, SRlfp, order=order, type='lfilt')
trace_filtfilt = lfp.butter_bandpass_filter(trace, lowcut, highcut, SRlfp, order=order, type='filtfilt')

In [5]:
## Peak-trough method (Belluscio et al. 2012 J Neuro) - fold into lfp_analysis.peak_trough_detect eventually
# from Nat's https://github.com/diba-lab/ephys/blob/master/Analysis/python/LFP/scripts/theta_phase_stim_verify.py

## Needs: trace, SRlfp, order
order = 2

lowcut_bell = 1  # Hz
highcut_bell = 80  # Hz
peak_trough_offset_sec = 0.07  # seconds to look for trough of wide-filtered trace next to 4-10Hz filtered trace

wide_filt = lfp.butter_bandpass_filter(trace, lowcut_bell, highcut_bell, SRlfp, order=order)

fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)
fig.set_size_inches([26, 3])
hraw = ax.plot(time_plot, trace)
ax.plot(time_plot, wide_filt, 'm')
ax.plot(time_plot, trace_lfilt, 'k--')
ax.set_xlim([start_time*60, start_time*60 + time_span])
ax.set_ylim([-v_range, v_range])
ax.set_xlabel(['Time (s)'])
ax.set_ylabel('uV')

offset_frames = np.round(peak_trough_offset_sec*SRlfp)

# First detect peak and trough off narrowband filtered signal - do hilbert transform
# trough = -pi->pi, peak = 0 (- -> +)
trace_analytic = signal.hilbert(trace_lfilt)  # get real and imaginary parts of signal
trig_trace_phase = np.angle(trace_analytic)
# ax.plot(time_plot, trig_trace_phase*v_range/8, 'r-')
peak_bool = np.bitwise_and(trig_trace_phase[0:-1] < 0, trig_trace_phase[1:] >= 0)
peak_bool = np.append(peak_bool, False)
trough_bool = np.bitwise_and(trig_trace_phase[0:-1] > 0, trig_trace_phase[1:] <= 0)
trough_bool = np.append(trough_bool, False)

# Indices to peak and trough of narrowband trace
peak_inds = np.where(peak_bool)[0]
trough_inds = np.where(trough_bool)[0]

# Check that above code works...
# ax.plot(time_plot[peak_bool], trace_lfilt[peak_bool], 'r*')
# ax.plot(time_plot[trough_bool], trace_lfilt[trough_bool], 'g*')

##  Plot times between peak and trough - seems likes looking 0.07 seconds to either side should be ok...
fig2, ax2 = plt.subplots(1, 2)
ax2[0].hist(np.diff(np.where(trough_bool))[0]/SRlfp)
ax2[0].set_xlabel('Trough-to-trough times (s)')
ax2[1].hist(np.diff(np.where(peak_bool))[0]/SRlfp)
ax2[1].set_xlabel('Peak-to-peak times (s)')

## now step through and find closest peak/trough in the wide-filtered trace when compared to the narrowband filtered trace.
# THIS IS ALL COMMENTED NOW SO THAT YOU DONT ACCIDENTALLY OVERWRITE EXISTING VALUES - NEED TO IMPLEMENT DOWNSAMPLING FIRST!!!
wide_peak_inds = []
wide_trough_inds = []

# Step through and look for each trough in the WIDE filtered signal between two peaks in the NARROW filtered signal
# how fast is this compared to just running it on all the trace and looking for closest inds? Bet it depends on if I
# downsample first...

n = 0
for idp, idp1 in zip(peak_inds[0:-1], peak_inds[1:]):
    wide_trough_inds.append(lfp.get_local_extrema(wide_filt[idp:idp1], type='min') + idp)
    n = n + 1
    if int(n/100) == n/100:
        print(n)

n = 0
# Ditto to above but for peaks
for idt, idt1 in zip(trough_inds[0:-1], trough_inds[1:]):
    wide_peak_inds.append(lfp.get_local_extrema(wide_filt[idt:idt1], type='max') + idt)
    n = n + 1
    if int(n/100) == n/100:
        print(n)

## looks decent except when there is crappy theta. Filter out these epochs? Put on speed threshold?
wide_peak_inds_good = [idp for idp in wide_peak_inds if not np.isnan(idp)]
wide_trough_inds_good = [idt for idt in wide_trough_inds if not np.isnan(idt)]

ax.plot(time_plot[wide_peak_inds_good], wide_filt[wide_peak_inds_good], 'ro')
ax.plot(time_plot[wide_trough_inds_good], wide_filt[wide_trough_inds_good], 'go')

## Get rise and falling times of theta - trough = -pi/+pi, peak = 0

# if peak times generally lead trough times, chop off first peak value
if np.nanmean(np.array(wide_peak_inds) - np.array(wide_trough_inds)) < 0:
    peak_inds_use = wide_peak_inds[1:]
    trough_inds_use = wide_trough_inds[0:-1]
    next_trough_inds = wide_trough_inds[1:]
else:
    peak_inds_use = wide_peak_inds
    trough_inds_use = wide_trough_inds
    next_trough_inds = wide_trough_inds[1:]


wave_phase_inds = []
wave_phases = []
for idt, idp, idt1 in zip(trough_inds_use, peak_inds_use, next_trough_inds):
    if not np.any(np.isnan([idt, idp, idt1])) and idt < idp < idt1:  # only run below if you have reliable peak/trough info
        trace_snippet = wide_filt[idt:idt1]  # grab a snippet of the trace to use
        if np.all(trace_snippet <= 0) or np.all(trace_snippet >= 0) or trace_snippet[0] > 0 or trace_snippet[-1] > 0\
                or wide_filt[idp] < 0:  # Make sure trace is not all above or below zero and that peak/troughs are above/below zero
            wave_phase_inds.extend([np.nan, np.nan, np.nan, np.nan, np.nan])
        else:
            rise_zero = np.max(np.where(np.bitwise_and(trace_snippet <= 0, np.arange(idt, idt1) < idp))[0])
            fall_zero = np.min(np.where(np.bitwise_and(trace_snippet <= 0, np.arange(idt, idt1) > idp))[0])
            wave_phase_inds.extend([idt, idt + rise_zero, idp, idt + fall_zero, idt1-1])
    else:
        wave_phase_inds.extend([np.nan, np.nan, np.nan, np.nan, np.nan])
    wave_phases.extend([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
    

NameError: name 'trace' is not defined

In [ ]:
# now plot
wave_phase_inds_good = [idph for idph in wave_phase_inds if not np.isnan(idph)]
wave_phases_good = [ph for ph, idph in zip(wave_phases, wave_phase_inds) if not np.isnan(idph)]
ax.plot(time_plot[wave_phase_inds_good], np.asarray(wave_phases_good)*v_range/8, 'r-')


## histogram of rise times vs fall times overlaid to prove I'm doing things correctly
fig35, ax35 = plt.subplots(1, 2)
fig35.set_size_inches([13.5, 4.8])
rise_times = (np.array(peak_inds_use) - np.array(trough_inds_use))/SRlfp
fall_times = (np.array(next_trough_inds) - np.array(peak_inds_use))/SRlfp
ax35[0].hist(rise_times, bins=20, range=(-0.15, 0.3))
ax35[0].set_title('Peak-Trough Method')
ax35[0].set_xlabel('Rising Phase Times (s)')
ax35[0].text(0.15, 1000, 'mean = ' + '{:.3f}'.format(np.nanmean(rise_times)) + ' sec')
ax35[1].hist(fall_times, bins=20, range=(-0.15, 0.3))
ax35[1].set_title('Peak-Trough Method')
ax35[1].set_xlabel('Falling Phase Times (s)')
ax35[1].text(0.15, 1000, 'mean = ' + '{:.3f}'.format(np.nanmean(fall_times)) + ' sec')
